In [1]:
import pandas as pd
import numpy as np
import math 
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# function to make datasets smaller
import sys

def return_size(df):
    """Return size of dataframe in gigabytes"""
    return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert objects to category
        if (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
    
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

In [4]:
# Unpack all the dataset
bureau = pd.read_csv(r"D:\Project Data Prep\Version 1\cleaned_bureau_ver1.csv")
credit_card_balance = pd.read_csv(r"D:\Project Data Prep\Version 1\dseb63_credit_card_balance_ver1 1.csv")
installment_payments = pd.read_csv(r"D:\Project Data Prep\Version 1\installment_payments_ver1 1.csv")
pos_cash = pd.read_csv(r"D:\Project Data Prep\Version 1\POS_CASH_CLEANED_xulioutlier_truoc_roi_fill_null ver 1.1.csv")
previous_application = pd.read_csv(r"D:\Project Data Prep\Version 1\previous_application ver1.csv")
bureau_balance = pd.read_csv(r"C:\Users\ADMIN\Downloads\dseb-64-data-preparation-final-project\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau_balance.csv")

In [5]:
bureau = convert_types(bureau)
credit_card_balance = convert_types(credit_card_balance)
installment_payments = convert_types(installment_payments)
pos_cash = convert_types(pos_cash)
previous_application = convert_types(previous_application)
bureau_balance = convert_types(bureau_balance)


# Feature Engineering

In [6]:
# Group by using SK_ID_CURR, then create columns based on informations about each applicants
num_aggregations = {
    'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['max', 'mean'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean', 'sum'],
    'AMT_CREDIT_SUM_LIMIT': ['max', 'mean', 'sum'],
    'AMT_ANNUITY': ['max', 'mean', 'sum'],
    'CNT_CREDIT_PROLONG': ['max', 'sum']}
bureau_agg = bureau.groupby('SK_ID_CURR').agg(num_aggregations)
bureau_agg.columns = pd.Index(['BUR_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()]) #Change the name of columns
bureau_agg.head()

,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_VAR,BUR_DAYS_CREDIT_ENDDATE_MIN,BUR_DAYS_CREDIT_ENDDATE_MAX,BUR_DAYS_CREDIT_ENDDATE_MEAN,BUR_DAYS_CREDIT_UPDATE_MAX,BUR_DAYS_CREDIT_UPDATE_MEAN,BUR_CREDIT_DAY_OVERDUE_MAX,...,BUR_AMT_CREDIT_SUM_OVERDUE_MEAN,BUR_AMT_CREDIT_SUM_OVERDUE_SUM,BUR_AMT_CREDIT_SUM_LIMIT_MAX,BUR_AMT_CREDIT_SUM_LIMIT_MEAN,BUR_AMT_CREDIT_SUM_LIMIT_SUM,BUR_AMT_ANNUITY_MAX,BUR_AMT_ANNUITY_MEAN,BUR_AMT_ANNUITY_SUM,BUR_CNT_CREDIT_PROLONG_MAX,BUR_CNT_CREDIT_PROLONG_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,-63,-63,-63.000000,NaN,237.0,237.0,237.000000,-28.0,-28.000000,0,...,0.0,0.0,0.0,0.0,0.0,8553.452148,8553.452148,8553.452148,0,0
1,-2348,-2348,-2348.000000,NaN,-2044.0,-2044.0,-2044.000000,-18.0,-18.000000,0,...,0.0,0.0,0.0,0.0,0.0,8553.452148,8553.452148,8553.452148,0,0
2,-2901,-30,-810.333333,1.222703e+06,-2597.0,274.0,-572.166687,-7.0,-566.916687,0,...,0.0,0.0,0.0,0.0,0.0,8553.452148,8553.452148,51320.710938,0,0
3,-2865,-116,-1131.428571,1.260530e+06,-2683.0,502.0,-658.714294,-7.0,-730.857117,0,...,0.0,0.0,0.0,0.0,0.0,8553.452148,8553.452148,59874.164062,0,0
4,-1056,-313,-773.333333,1.082075e+05,-509.0,1148.0,474.500000,-6.0,-63.333332,0,...,0.0,0.0,0.0,0.0,0.0,8553.452148,8553.452148,51320.710938,0,0


In [7]:
# Adding aggregates for closed credit
closed = bureau[bureau['CREDIT_ACTIVE'] == 'Closed']
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
bur_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
bur_agg.head()

,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_VAR,BUR_DAYS_CREDIT_ENDDATE_MIN,BUR_DAYS_CREDIT_ENDDATE_MAX,BUR_DAYS_CREDIT_ENDDATE_MEAN,BUR_DAYS_CREDIT_UPDATE_MAX,BUR_DAYS_CREDIT_UPDATE_MEAN,BUR_CREDIT_DAY_OVERDUE_MAX,...,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_OVERDUE_SUM,CLOSED_AMT_CREDIT_SUM_LIMIT_MAX,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_AMT_ANNUITY_MAX,CLOSED_AMT_ANNUITY_MEAN,CLOSED_AMT_ANNUITY_SUM,CLOSED_CNT_CREDIT_PROLONG_MAX,CLOSED_CNT_CREDIT_PROLONG_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,-63,-63,-63.000000,NaN,237.0,237.0,237.000000,-28.0,-28.000000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2348,-2348,-2348.000000,NaN,-2044.0,-2044.0,-2044.000000,-18.0,-18.000000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2901,-30,-810.333333,1.222703e+06,-2597.0,274.0,-572.166687,-7.0,-566.916687,0,...,0.0,0.0,0.0,0.0,0.0,8553.452148,8553.452148,34213.808594,0.0,0.0
3,-2865,-116,-1131.428571,1.260530e+06,-2683.0,502.0,-658.714294,-7.0,-730.857117,0,...,0.0,0.0,0.0,0.0,0.0,8553.452148,8553.452148,34213.808594,0.0,0.0
4,-1056,-313,-773.333333,1.082075e+05,-509.0,1148.0,474.500000,-6.0,-63.333332,0,...,0.0,0.0,0.0,0.0,0.0,8553.452148,8553.452148,25660.355469,0.0,0.0


bureau_balance

In [8]:
# Aggregation for bureau balance
bb_aggregations = {'MONTHS_BALANCE': ['min', 'mean', 'max', 'size']}
bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(bb_aggregations)
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()]) # Change columns name
bb_agg.head()

,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE
SK_ID_BUREAU,,,,
5001709,-96,-48.0,0,97
5001710,-82,-41.0,0,83
5001711,-3,-1.5,0,4
5001712,-18,-9.0,0,19
5001713,-21,-10.5,0,22


CREDIT_CARD_BALANCE

In [9]:
# Aggregations for credit card balance
cred_aggregations = {'MONTHS_BALANCE': ['min', 'mean', 'max'],
                    'AMT_BALANCE': ['min', 'mean', 'max'],
                    'AMT_DRAWINGS_ATM_CURRENT': ['max', 'sum', 'mean'],
                    'AMT_DRAWINGS_OTHER_CURRENT': ['max', 'sum', 'mean', 'size'],
                    'AMT_PAYMENT_CURRENT': ['min', 'max', 'sum', 'mean', 'size'],
                    'AMT_RECEIVABLE': ['max', 'sum', 'mean'],
                    'CNT_DRAWINGS_ATM_CURRENT': ['sum'],
                    'CNT_INSTALLMENT_MATURE_CUM': ['max', 'sum']}
cred_agg = credit_card_balance.groupby('SK_ID_CURR').agg(cred_aggregations)
cred_agg.columns = pd.Index(['CRED_' + e[0] + "_" + e[1].upper() for e in cred_agg.columns.tolist()])
# Count of total entries
cred_agg['CRED_COUNT'] = credit_card_balance.groupby('SK_ID_CURR').size()
cred_agg.head()

,CRED_MONTHS_BALANCE_MIN,CRED_MONTHS_BALANCE_MEAN,CRED_MONTHS_BALANCE_MAX,CRED_AMT_BALANCE_MIN,CRED_AMT_BALANCE_MEAN,CRED_AMT_BALANCE_MAX,CRED_AMT_DRAWINGS_ATM_CURRENT_MAX,CRED_AMT_DRAWINGS_ATM_CURRENT_SUM,CRED_AMT_DRAWINGS_ATM_CURRENT_MEAN,CRED_AMT_DRAWINGS_OTHER_CURRENT_MAX,...,CRED_AMT_PAYMENT_CURRENT_SUM,CRED_AMT_PAYMENT_CURRENT_MEAN,CRED_AMT_PAYMENT_CURRENT_SIZE,CRED_AMT_RECEIVABLE_MAX,CRED_AMT_RECEIVABLE_SUM,CRED_AMT_RECEIVABLE_MEAN,CRED_CNT_DRAWINGS_ATM_CURRENT_SUM,CRED_CNT_INSTALLMENT_MATURE_CUM_MAX,CRED_CNT_INSTALLMENT_MATURE_CUM_SUM,CRED_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,-8,-4.5,-1,0.000000,6735.504395,36569.429688,0.000000,0.000000,0.000000,0.000000,...,64661.273438,8082.659180,8,36569.429688,5.388404e+04,6735.504395,0,1,2,8
1,-9,-5.0,-1,15271.200195,147012.828125,226013.843750,180000.000000,234000.000000,26000.000000,0.000000,...,67220.203125,7468.911621,9,224385.203125,1.317009e+06,146334.375000,7,8,36,9
3,-11,-6.0,-1,0.000000,124425.875000,140704.968750,6024.271484,6024.271484,547.661072,298.488251,...,106405.343750,9673.212891,11,138996.984375,1.353635e+06,123057.742188,0,9,45,11
7,-18,-9.5,-1,0.000000,0.000000,0.000000,6024.271484,108436.890625,6024.271484,298.488251,...,104313.601562,5795.200195,18,0.000000,0.000000e+00,0.000000,0,0,0,18
9,-11,-6.0,-1,0.000000,428.318176,4711.500000,68400.000000,68400.000000,6218.181641,0.000000,...,27288.583984,2480.780273,11,0.000000,0.000000e+00,0.000000,3,1,10,11


INSTALLMENT_PAYMENT

In [10]:
# Finding percentage and difference paid in each installment
installment_payments['PAYMENT_PERC'] = installment_payments['AMT_PAYMENT'] / installment_payments['AMT_INSTALLMENT']
installment_payments['PAYMENT_DIFF'] = installment_payments['AMT_INSTALLMENT'] - installment_payments['AMT_PAYMENT']
# Finding days past due (DPD) and days before due (DBD)
installment_payments['DPD'] = installment_payments['DAYS_ENTRY_PAYMENT'] - installment_payments['DAYS_INSTALLMENT']
installment_payments['DBD'] = installment_payments['DAYS_INSTALLMENT'] - installment_payments['DAYS_ENTRY_PAYMENT']
installment_payments['DPD'] = installment_payments['DPD'].apply(lambda x: x if x > 0 else 0)
installment_payments['DBD'] = installment_payments['DBD'].apply(lambda x: x if x > 0 else 0)
# Creating aggregations
inst_aggregations = {
    'NUM_INSTALLMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALLMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum'],
    'NUM_INSTALLMENT_VERSION': ['max']}
inst_agg = installment_payments.groupby('SK_ID_CURR').agg(inst_aggregations)
inst_agg.columns = pd.Index(['INST_' + e[0] + "_" + e[1].upper() for e in inst_agg.columns.tolist()])
# Count of installments accounts
inst_agg['INST_COUNT'] = installment_payments.groupby('SK_ID_CURR').size()
inst_agg.head()

,INST_NUM_INSTALLMENT_VERSION_MAX,INST_DPD_MAX,INST_DPD_MEAN,INST_DPD_SUM,INST_DBD_MAX,INST_DBD_MEAN,INST_DBD_SUM,INST_PAYMENT_PERC_MAX,INST_PAYMENT_PERC_MEAN,INST_PAYMENT_PERC_SUM,...,INST_AMT_INSTALLMENT_MEAN,INST_AMT_INSTALLMENT_SUM,INST_AMT_PAYMENT_MIN,INST_AMT_PAYMENT_MAX,INST_AMT_PAYMENT_MEAN,INST_AMT_PAYMENT_SUM,INST_DAYS_ENTRY_PAYMENT_MAX,INST_DAYS_ENTRY_PAYMENT_MEAN,INST_DAYS_ENTRY_PAYMENT_SUM,INST_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,2,0,0.000,0,64,10.900000,327,1.00000,0.998154,29.944622,...,7585.663574,227569.906250,141.570007,34312.500000,7521.133301,225634.000000,-5,-150.433333,-4513,30
1,2,29,2.075,83,30,4.000000,160,1.00000,0.849539,33.981541,...,15427.903320,617116.125000,120.779999,34312.500000,11512.316406,460492.656250,-18,-622.050000,-24882,40
3,2,0,0.000,0,99,11.581395,498,2.51059,1.050096,45.154121,...,5503.119141,236634.125000,78.434998,34312.500000,5692.181641,244763.812500,-5,-228.232558,-9814,43
4,1,0,0.000,0,23,17.166667,103,1.00000,1.000000,6.000000,...,8953.522461,53721.132812,8933.985352,8957.429688,8953.522461,53721.132812,-2230,-2299.166667,-13795,6
5,1,0,0.000,0,17,9.200000,92,1.00000,1.000000,10.000000,...,9818.644531,98186.445312,9799.650391,9820.754883,9818.644531,98186.445312,-1808,-1944.200000,-19442,10


PREVIOUS APPLICATION

In [11]:
# Percentage of value ask / value received percentage
previous_application['APP_CREDIT_PERC'] = previous_application['AMT_APPLICATION'] / previous_application['AMT_CREDIT']
# Aggregations for previous applications
prev_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean', 'sum'],
    'AMT_APPLICATION': ['min', 'max', 'mean', 'sum'],
    'AMT_CREDIT': ['min', 'max', 'mean', 'sum'],
    'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum', 'size'],
    'RATE_INTEREST_PRIMARY': ['max'],
    'RATE_DOWN_PAYMENT': ['max'],
    'DAYS_LAST_DUE': ['max', 'min'],
    'DAYS_TERMINATION': ['max', 'min']}
prev_agg = previous_application.groupby('SK_ID_CURR').agg(prev_aggregations)
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
# Previous Applications: Approved Applications only
approved = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved']
approved_agg = approved.groupby('SK_ID_CURR').agg(prev_aggregations)
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
# Previous Applications: Refused Applications - only numerical features
refused = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused']
refused_agg = refused.groupby('SK_ID_CURR').agg(prev_aggregations)
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
prev_agg.head()

,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_ANNUITY_SUM,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_APPLICATION_SUM,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,...,REFUSED_DAYS_DECISION_MAX,REFUSED_DAYS_DECISION_MEAN,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM,REFUSED_CNT_PAYMENT_SIZE,REFUSED_RATE_INTEREST_PRIMARY_MAX,REFUSED_DAYS_LAST_DUE_MAX,REFUSED_DAYS_LAST_DUE_MIN,REFUSED_DAYS_TERMINATION_MAX,REFUSED_DAYS_TERMINATION_MIN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,2250.000000,11180.384766,6659.137207,39954.824219,25200.0,120658.5,78399.000000,470394.0,21928.5,120658.5,...,-121.0,-121.0,24.0,24.0,1.0,0.176342,180792.0,180792.0,194000.0,194000.0
1,2250.000000,25996.365234,13688.303711,95818.125000,0.0,337500.0,68303.570312,478125.0,0.0,384277.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2457.675049,24577.425781,9705.645508,38822.582031,16420.5,225000.0,85230.125000,340920.5,14170.5,239850.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3404.520020,14272.701172,8013.459473,48080.757812,0.0,135000.0,41482.500000,248895.0,0.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8957.429688,8957.429688,8957.429688,8957.429688,47250.0,47250.0,47250.000000,47250.0,44631.0,44631.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Aggregations for POS
pos_aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean'],
    'CNT_INSTALMENT_FUTURE': ['max', 'sum'],
    'CNT_INSTALMENT': ['max', 'sum']}
pos_agg = pos_cash.groupby('SK_ID_CURR').agg(pos_aggregations)
pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
# Count of pos cash accounts
pos_agg['POS_COUNT'] = pos_cash.groupby('SK_ID_CURR').size()
pos_agg.head()

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_CNT_INSTALMENT_FUTURE_MAX,POS_CNT_INSTALMENT_FUTURE_SUM,POS_CNT_INSTALMENT_MAX,POS_CNT_INSTALMENT_SUM,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,
0,-1.0,-7.466667,15,0,0.0,0,0.0,12.0,96.0,12.0,122.0,15
1,-12.0,-32.086956,23,0,0.0,0,0.0,30.5,378.0,36.0,517.0,23
2,-23.0,-55.147060,34,0,0.0,0,0.0,12.0,145.0,12.0,292.0,34
3,-5.0,-12.428572,14,0,0.0,0,0.0,12.0,111.0,12.0,139.0,14
4,-73.0,-76.000000,7,0,0.0,0,0.0,6.0,21.0,6.0,42.0,7


## Remove spare tables to have more memory

In [13]:
import gc
gc.enable()
del bureau, credit_card_balance, pos_cash, installment_payments, previous_application, bureau_balance,closed, closed_agg, approved, approved_agg
gc.collect()

0

# Joining Features

In [ ]:
train = pd.read_csv(r"D:\Project Data Prep\Version 2\EDA and Clean Train Dataset.csv")

In [15]:
train = convert_types(train)

In [16]:
#Create a few new features that might be helpful: 
#Annuity/income ratio
#Goods price/AMT credit
# AMT_Credit/AMT_Income_total
train['Ratio_Ann_Inc']=train['AMT_ANNUITY']/train['AMT_INCOME_TOTAL']
train['Ratio_Goods_Credit']=train['AMT_GOODS_PRICE']/train['AMT_CREDIT']
train['Ratio_Credit_Inc']=train['AMT_CREDIT']/train['AMT_INCOME_TOTAL']
train['CREDIT_TERM']=train['AMT_ANNUITY']/train['AMT_CREDIT']
train['DAYS_EMPLOYED_PERCENT'] = train['DAYS_EMPLOYED'] / train['DAYS_BIRTH']


In [17]:
train = train.join(bur_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(bb_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(cred_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(inst_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(prev_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(pos_agg, how = "left", on = 'SK_ID_CURR')
train.head()

,Unnamed: 0,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_CNT_INSTALMENT_FUTURE_MAX,POS_CNT_INSTALMENT_FUTURE_SUM,POS_CNT_INSTALMENT_MAX,POS_CNT_INSTALMENT_SUM,POS_COUNT
0,0,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,28.0,0.0,0.0,0.0,0.0,12.0,162.000000,12.0,283.0,28.0
1,1,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,21.0,0.0,0.0,0.0,0.0,30.5,165.204803,45.0,249.0,21.0
2,2,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,66.0,0.0,0.0,0.0,0.0,24.0,592.000000,24.0,1012.0,66.0
3,3,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,83.0,0.0,0.0,0.0,0.0,30.0,341.000000,30.0,956.0,83.0
4,4,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,...,11.0,0.0,0.0,0.0,0.0,10.0,55.000000,10.0,110.0,11.0


# Clean the training and test dataset

In [ ]:
# Read test file
test = pd.read_csv(r'D:\Project Data Prep\Version 2\EDA and Clean Test Dataset.csv')

In [19]:
test = convert_types(test)

In [20]:
test['Ratio_Ann_Inc']=test['AMT_ANNUITY']/test['AMT_INCOME_TOTAL']
test['Ratio_Goods_Credit']=test['AMT_GOODS_PRICE']/test['AMT_CREDIT']
test['Ratio_Credit_Inc']=test['AMT_CREDIT']/test['AMT_INCOME_TOTAL']
test['CREDIT_TERM']=test['AMT_ANNUITY']/test['AMT_CREDIT']
test['DAYS_EMPLOYED_PERCENT'] = test['DAYS_EMPLOYED'] / test['DAYS_BIRTH']

In [21]:
# drop train columns that contains all null values
train.dropna(axis = 1, how = 'all', inplace = True)

In [22]:
test = test.join(bur_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(bb_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(cred_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(inst_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(prev_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(pos_agg, how = "left", on = 'SK_ID_CURR')

In [23]:
del bur_agg, bb_agg, cred_agg, inst_agg, prev_agg, pos_agg

In [24]:
# Drop colums that contain all null values
test.dropna(axis = 1, how = 'all', inplace = True)

In [25]:
# Drop unwanted column
test.drop('Unnamed: 0', axis = 1, inplace = True)

In [26]:
# Choose training set x and target y

y = train['TARGET']
x = train.iloc[:, 2:]
x.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_CNT_INSTALMENT_FUTURE_MAX,POS_CNT_INSTALMENT_FUTURE_SUM,POS_CNT_INSTALMENT_MAX,POS_CNT_INSTALMENT_SUM,POS_COUNT
0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,28.0,0.0,0.0,0.0,0.0,12.0,162.000000,12.0,283.0,28.0
1,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,21.0,0.0,0.0,0.0,0.0,30.5,165.204803,45.0,249.0,21.0
2,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,66.0,0.0,0.0,0.0,0.0,24.0,592.000000,24.0,1012.0,66.0
3,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,"Spouse, partner",...,83.0,0.0,0.0,0.0,0.0,30.0,341.000000,30.0,956.0,83.0
4,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,1530000.0,Unaccompanied,...,11.0,0.0,0.0,0.0,0.0,10.0,55.000000,10.0,110.0,11.0


In [27]:
test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_CNT_INSTALMENT_FUTURE_MAX,POS_CNT_INSTALMENT_FUTURE_SUM,POS_CNT_INSTALMENT_MAX,POS_CNT_INSTALMENT_SUM,POS_COUNT
0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,Unaccompanied,...,23.0,0.0,0.0,0.0,0.0,12.0,133.0,12.0,256.0,23.0
1,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,Unaccompanied,...,44.0,0.0,0.0,0.0,0.0,12.0,195.0,12.0,333.0,44.0
2,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,Unaccompanied,...,21.0,0.0,0.0,0.0,0.0,10.0,95.0,10.0,190.0,21.0
3,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,Unaccompanied,...,25.0,0.0,0.0,0.0,0.0,12.0,78.0,12.0,300.0,25.0
4,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,234000.0,Unaccompanied,...,26.0,0.0,0.0,0.0,0.0,12.0,148.0,12.0,245.0,26.0


In [28]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [29]:
# Replace infinite values with null values
test.replace([np.inf, -np.inf], np.nan, inplace=True)

## Fill mising values in train and test dataset

In [36]:
# Fill missing values in both train and test dataset
for col in x.columns:
    if x[col].dtype in ['object', 'category']:
        imputer = SimpleImputer(strategy= 'most_frequent')
        imputer.fit(x[[col]])
        x[col] = imputer.transform(x[[col]]).ravel()
        test[col] = imputer.transform(test[[col]]).ravel()
    else:
        imputer = SimpleImputer(strategy= 'mean')
        imputer.fit(x[[col]])
        x[col] = imputer.transform(x[[col]]).ravel()
        test[col] = imputer.transform(test[[col]]).ravel()
        



In [37]:
onehot = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # Set sparse=False to get a dense array

# For the training dataset
categorical_columns = [col for col in x.columns if x[col].dtype in ['object', 'category']]  # Identify categorical columns
x_encoded = onehot.fit_transform(x[categorical_columns])  # Fit and transform only the categorical columns

# Replace the categorical columns with the one-hot encoded version
x = pd.concat([
    x.drop(columns=categorical_columns).reset_index(drop=True),  # Drop original categorical columns
    pd.DataFrame(x_encoded, columns=onehot.get_feature_names_out(categorical_columns))  # Add encoded columns
], axis=1)

# For the test dataset
test_encoded = onehot.transform(test[categorical_columns])  # Transform the test dataset with the same encoder

# Replace the categorical columns in the test dataset
test = pd.concat([
    test.drop(columns=categorical_columns).reset_index(drop=True),  # Drop original categorical columns
    pd.DataFrame(test_encoded, columns=onehot.get_feature_names_out(categorical_columns))  # Add encoded columns
], axis=1)

## Polynomial Features

In [38]:
# Create polynomial features using the EXT_SOURCE variables and the DAYS_BIRTH variable.
# use polynomials with degree of 3
# Make a new dataframe for polynomial features
poly_features = x[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
poly_features_test = test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]


from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

In [39]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

Polynomial Features shape:  (246009, 35)


In [40]:
# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, columns = poly_transformer.get_feature_names_out(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']))
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, columns = poly_transformer.get_feature_names_out(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Merge polynomial features into training dataframe
poly_features['SK_ID_CURR'] = x['SK_ID_CURR']
x = x.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = test['SK_ID_CURR']
test = test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')


# Print out the new shapes
print('Training data with polynomial features shape: ', x.shape)
print('Testing data with polynomial features shape:  ', test.shape)


Training data with polynomial features shape:  (246009, 519)
Testing data with polynomial features shape:   (61502, 519)


# Tuning hyperparameters

In [42]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold


In [66]:
from sklearn.metrics import roc_auc_score

In [ ]:
# Scale the train and test dataset
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(x)
x_scale = min_max_scaler.transform(x)
test_scale = min_max_scaler.transform(test)

In [70]:
# Slpit the dataset in to training dataset and validation dataset
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size= 0.01)

In [46]:
# Hyperparameters of logistic regresion
params = {
    'penalty': ['l2'],
    'C': [0.001, 0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'saga'],
    'class_weight': ['balanced', None]
}

kf = StratifiedKFold(n_splits=5, shuffle=False)

In [ ]:
# Tuning hyperparameters
model = LogisticRegression(max_iter=2000, tol = 1e-4)
grid_search = GridSearchCV(model, params, cv = kf, scoring= 'roc_auc')
grid_search.fit(x_val, y_val)

print("Best Parameters:", grid_search.best_params_)


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the 

Best Parameters: {'C': 0.1, 'class_weight': None, 'penalty': 'l2', 'solver': 'saga'}


In [61]:
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'C': 0.1, 'class_weight': None, 'penalty': 'l2', 'solver': 'saga'}


In [ ]:
#Build model
true_model = LogisticRegression(max_iter= 3000, solver= 'saga', penalty= 'l2', class_weight= 'balanced', C = 0.1, tol = 0.0001)
true_model.fit(x_train, y_train)
result = true_model.predict_proba(test_scale)

In [76]:
submission_ver2 = pd.DataFrame({"SK_ID_CURR": test['SK_ID_CURR'].astype('int'), "TARGET": result[:, 1]})

In [69]:
roc_auc_score(y_val, result)

0.693262835408657

In [77]:
# #Output result
submission_ver2.to_csv(r'D:\Project Data Prep\Version 2\submission_ver4_final.csv', index=False , header = 1)